In [18]:
import numpy as np

## Input Parameters

In [19]:
# geometry
L = 1.0
w = 0.05
h = 0.1

# mesh
shape = (100, 10, 5)

# material
E = 1000
nu = 0.2

# load
F = 100

## Analytic Bernoulli Solution

In [20]:
EI = E * w * h**3 / 12
UY_Bernoulli = -F * L**3 / (3 * EI)

## Finite Element Solution

In [21]:
# Hooke model
A = np.array([
    [1, nu, nu, 0, 0, 0], 
    [nu, 1, nu, 0, 0, 0],
    [nu, nu, 1, 0, 0, 0], 
    [0., 0, 0, (1-nu)/2, 0, 0],
    [0., 0, 0, 0, (1-nu)/2, 0],
    [0., 0, 0, 0, 0, (1-nu)/2]]) * (E /(1-nu**2))

In [22]:
from sigmaepsilon.mesh import TriMesh
from sigmaepsilon.mesh.extrude import extrude_T3_TET4
trimesh = TriMesh(size=(L, h), shape=shape)
coords = trimesh.coords()
topo = trimesh.topology()
coords, topo = extrude_T3_TET4(coords, topo, w, shape[-1])

In [23]:
# essential boundary conditions
cond_ebc = np.abs(coords[:, 0]) <= 1e-5
ebcinds = np.where(cond_ebc)[0]
fixity = np.zeros((coords.shape[0], 3), dtype=bool)
fixity[ebcinds, :] = True

In [24]:
# natural boundary conditions
target = np.array([L, h/2, 0])
nbcinds = [trimesh.index_of_closest_point(target)]
loads = np.zeros((coords.shape[0], 3))
loads[nbcinds, 1] = -F

In [25]:
from sigmaepsilon.solid.fem import FemMesh, Structure, TET4
mesh = FemMesh(coords=coords, topo=topo, celltype=TET4, 
               fixity=fixity, loads=loads, model=A)
structure = Structure(mesh=mesh)

In [26]:
structure.linsolve()
structure.summary

{'preproc': {'sparsify': False},
 'proc': {'time [ms]': 8.000612258911133,
  'N': 18000,
  'use_umfpack': True,
  'permc_spec': 'COLAMD',
  'solver': 'pardiso'},
 'postproc': {},
 'number of elements': 26730,
 'number of nodes': 6000,
 'dofs per node': 3}

In [27]:
UY_FEM = structure.pointdata.dofsol.to_numpy()[nbcinds[0]][1]

## Comparison

In [28]:
print("Analytic Bernoulli Solution : {}".format(UY_Bernoulli))
print("FEM Solution : {}".format(UY_FEM))
diff = 100 * (UY_FEM - UY_Bernoulli) / UY_Bernoulli
print("Difference : {} %".format(diff))
structure.plot()

Analytic Bernoulli Solution : -7999.999999999998
FEM Solution : -8182.466469034602
Difference : 2.2808308629325444 %
